<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/extract_weights_mnet_260_640_80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
#!pip uninstall numpy -y
#!pip install numpy==1.14.5
!pip install numpy
!pip install -U -q PyDrive
!pip install git+https://github.com/justadudewhohacks/image_augment.py
!pip install git+https://github.com/justadudewhohacks/colabsnippets
!pip install albumentations==0.3.3

  Cloning https://github.com/justadudewhohacks/image_augment.py to /tmp/pip-req-build-vju9x81i
  Running command git clone -q https://github.com/justadudewhohacks/image_augment.py /tmp/pip-req-build-vju9x81i
  Created wheel for augment: filename=augment-0.0.5-cp36-none-any.whl size=4879 sha256=c1d9b761d6d3d04e74a00bc3020a1cae6951fd3c6e9f982abe4b2436514d5934
  Stored in directory: /tmp/pip-ephem-wheel-cache-i2a27iml/wheels/6f/f7/4a/e8e6dc3a68ae11e9b1f6872c5c1be9bed2052f05e487df549f
Successfully built augment
  Cloning https://github.com/justadudewhohacks/colabsnippets to /tmp/pip-req-build-d28x7c6w
  Running command git clone -q https://github.com/justadudewhohacks/colabsnippets /tmp/pip-req-build-d28x7c6w
  Created wheel for colabsnippets: filename=colabsnippets-0.0.114-cp36-none-any.whl size=73195 sha256=2c082920d88bf444c0ec6eb91a2a508a82400ea65f752340814f5057229e0547
  Stored in directory: /tmp/pip-ephem-wheel-cache-_oi797t1/wheels/a4/f3/4d/514be480c5a7dfea589012f45748e962bfe35639211

# ULFGFD+


In [2]:
!git clone https://github.com/biubug6/Face-Detector-1MB-with-landmark/

Cloning into 'Face-Detector-1MB-with-landmark'...
remote: Enumerating objects: 129, done.
remote: Total 129 (delta 0), reused 0 (delta 0), pack-reused 129
Receiving objects: 100% (129/129), 7.87 MiB | 10.53 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [0]:
import sys
sys.path.append('/content/Face-Detector-1MB-with-landmark')

In [0]:
!pip install torchviz

In [0]:
from __future__ import print_function
import os
import argparse
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import time
from data import cfg_mnet, cfg_slim, cfg_rfb
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
import cv2

from models.retinaface import RetinaFace
from models.net_slim import Slim
from models.net_rfb import RFB
from utils.box_utils import decode, decode_landm
from utils.timer import Timer

def check_keys(model, pretrained_state_dict):
    ckpt_keys = set(pretrained_state_dict.keys())
    model_keys = set(model.state_dict().keys())
    used_pretrained_keys = model_keys & ckpt_keys
    unused_pretrained_keys = ckpt_keys - model_keys
    missing_keys = model_keys - ckpt_keys
    print('Missing keys:{}'.format(len(missing_keys)))
    print('Unused checkpoint keys:{}'.format(len(unused_pretrained_keys)))
    print('Used keys:{}'.format(len(used_pretrained_keys)))
    assert len(used_pretrained_keys) > 0, 'load NONE from pretrained checkpoint'
    return True


def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}


def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))
    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')
    check_keys(model, pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    return model

import torch
import torch.nn as nn
import torchvision.models.detection.backbone_utils as backbone_utils
import torchvision.models._utils as _utils
import torch.nn.functional as F
from collections import OrderedDict

def conv_bn(inp, oup, stride = 1):
  return nn.Sequential(
    nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
    nn.BatchNorm2d(oup),
    nn.ReLU(inplace=True)
  )

def depth_conv2d(inp, oup, kernel=1, stride=1, pad=0):
  return nn.Sequential(
    nn.Conv2d(inp, inp, kernel_size = kernel, stride = stride, padding=pad, groups=inp),
    nn.ReLU(inplace=True),
    nn.Conv2d(inp, oup, kernel_size=1)
  )

def conv_dw(inp, oup, stride):
  return nn.Sequential(
    nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
    nn.BatchNorm2d(inp),
    nn.ReLU(inplace=True),

    nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
    nn.BatchNorm2d(oup),
    nn.ReLU(inplace=True)
  )

def conv_bn1X1(inp, oup, stride):
  return nn.Sequential(
      nn.Conv2d(inp, oup, 1, stride, padding=0, bias=False),
      nn.BatchNorm2d(oup),
      nn.ReLU(inplace=True)
  )

class FPN5(nn.Module):
  def __init__(self,in_channels_list,out_channels):
      super(FPN5,self).__init__()
      leaky = 0
      if (out_channels <= 64):
          leaky = 0.1
      self.output1 = conv_bn1X1(in_channels_list[0], out_channels, stride = 1)
      self.output2 = conv_bn1X1(in_channels_list[1], out_channels, stride = 1)
      self.output3 = conv_bn1X1(in_channels_list[2], out_channels, stride = 1)
      self.output4 = conv_bn1X1(in_channels_list[3], out_channels, stride = 1)
      self.output5 = conv_bn1X1(in_channels_list[4], out_channels, stride = 1)

      self.merge1 = conv_bn(out_channels, out_channels)
      self.merge2 = conv_bn(out_channels, out_channels)
      self.merge3 = conv_bn(out_channels, out_channels)
      self.merge4 = conv_bn(out_channels, out_channels)

  def forward(self, input):
      # names = list(input.keys())

      output1 = self.output1(input[0])
      output2 = self.output2(input[1])
      output3 = self.output3(input[2])
      output4 = self.output4(input[3])
      output5 = self.output5(input[4])

      
      up5 = F.interpolate(output5, size=[output4.size(2), output4.size(3)], mode="nearest")
      output4 = output4 + up5
      output4 = self.merge4(output4)

      up4 = F.interpolate(output4, size=[output3.size(2), output3.size(3)], mode="nearest")
      output3 = output3 + up4
      output3 = self.merge3(output3)

      up3 = F.interpolate(output3, size=[output2.size(2), output2.size(3)], mode="nearest")
      output2 = output2 + up3
      output2 = self.merge2(output2)

      up2 = F.interpolate(output2, size=[output1.size(2), output1.size(3)], mode="nearest")
      output1 = output1 + up2
      output1 = self.merge1(output1)

      out = [output1, output2, output3, output4, output5]
      return out

class FPN5_dw(nn.Module):
  def __init__(self,in_channels_list,out_channels):
      super(FPN5_dw,self).__init__()
      leaky = 0
      if (out_channels <= 64):
          leaky = 0.1
      self.output1 = conv_bn1X1(in_channels_list[0], out_channels, stride = 1)
      self.output2 = conv_bn1X1(in_channels_list[1], out_channels, stride = 1)
      self.output3 = conv_bn1X1(in_channels_list[2], out_channels, stride = 1)
      self.output4 = conv_bn1X1(in_channels_list[3], out_channels, stride = 1)
      self.output5 = conv_bn1X1(in_channels_list[4], out_channels, stride = 1)

      self.merge1 = conv_dw(out_channels, out_channels, 1)
      self.merge2 = conv_dw(out_channels, out_channels, 1)
      self.merge3 = conv_dw(out_channels, out_channels, 1)
      self.merge4 = conv_dw(out_channels, out_channels, 1)

  def forward(self, input):
      # names = list(input.keys())

      output1 = self.output1(input[0])
      output2 = self.output2(input[1])
      output3 = self.output3(input[2])
      output4 = self.output4(input[3])
      output5 = self.output5(input[4])

      
      up5 = F.interpolate(output5, size=[output4.size(2), output4.size(3)], mode="nearest")
      output4 = output4 + up5
      output4 = self.merge4(output4)

      up4 = F.interpolate(output4, size=[output3.size(2), output3.size(3)], mode="nearest")
      output3 = output3 + up4
      output3 = self.merge3(output3)

      up3 = F.interpolate(output3, size=[output2.size(2), output2.size(3)], mode="nearest")
      output2 = output2 + up3
      output2 = self.merge2(output2)

      up2 = F.interpolate(output2, size=[output1.size(2), output1.size(3)], mode="nearest")
      output1 = output1 + up2
      output1 = self.merge1(output1)

      out = [output1, output2, output3, output4, output5]
      return out

### A

In [0]:
class Slim_c64(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_c64, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 64, 2)
    self.conv10 = conv_dw(64, 64, 1)
    self.conv11 = conv_dw(64, 64, 1)

    self.conv12 = conv_dw(64, 64, 2)
    self.conv13 = conv_dw(64, 64, 1)

    self.conv14 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 64, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.loc, self.conf, self.landm = self.multibox(self.num_classes);

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 3 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 3 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 3 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [nn.Conv2d(64, 3 * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(64, 3 * num_classes, kernel_size=3, padding=1)]
    landm_layers += [nn.Conv2d(64, 3 * 10, kernel_size=3, padding=1)]
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14= self.conv14(x13)
    detections.append(x14)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

class Slim_c64_a5_buggy(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_c64_a5, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 64, 2)
    self.conv10 = conv_dw(64, 64, 1)
    self.conv11 = conv_dw(64, 64, 1)

    self.conv12 = conv_dw(64, 64, 2)
    self.conv13 = conv_dw(64, 64, 1)

    self.conv14 = conv_dw(64, 64, 2)
    self.conv15 = conv_dw(64, 64, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 64, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.loc, self.conf, self.landm = self.multibox(self.num_classes);

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [nn.Conv2d(64, 1 * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(64, 1 * num_classes, kernel_size=3, padding=1)]
    landm_layers += [nn.Conv2d(64, 1 * 10, kernel_size=3, padding=1)]

    loc_layers += [nn.Conv2d(64, 1 * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(64, 1 * num_classes, kernel_size=3, padding=1)]
    landm_layers += [nn.Conv2d(64, 1 * 10, kernel_size=3, padding=1)]
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv12(x13)
    x15 = self.conv13(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

class Slim_c64_a5(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_c64_a5, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 64, 2)
    self.conv10 = conv_dw(64, 64, 1)
    self.conv11 = conv_dw(64, 64, 1)

    self.conv12 = conv_dw(64, 64, 2)
    self.conv13 = conv_dw(64, 64, 1)

    self.conv14 = conv_dw(64, 64, 2)
    self.conv15 = conv_dw(64, 64, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 64, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.loc, self.conf, self.landm = self.multibox(self.num_classes);

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [nn.Conv2d(64, 1 * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(64, 1 * num_classes, kernel_size=3, padding=1)]
    landm_layers += [nn.Conv2d(64, 1 * 10, kernel_size=3, padding=1)]

    loc_layers += [nn.Conv2d(64, 1 * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(64, 1 * num_classes, kernel_size=3, padding=1)]
    landm_layers += [nn.Conv2d(64, 1 * 10, kernel_size=3, padding=1)]
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv16(x15)
    detections.append(x16)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

class Slim_a5(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_a5, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 128, 2)
    self.conv10 = conv_dw(128, 128, 1)
    self.conv11 = conv_dw(128, 128, 1)

    self.conv12 = conv_dw(128, 256, 2)
    self.conv13 = conv_dw(256, 256, 1)

    self.conv14 = conv_dw(256, 256, 2)
    self.conv15 = conv_dw(256, 256, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 256, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.loc, self.conf, self.landm = self.multibox(self.num_classes);

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(128, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(128, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(128, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 1 * 10, kernel=3, pad=1)]

    loc_layers += [nn.Conv2d(256, 1 * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, 1 * num_classes, kernel_size=3, padding=1)]
    landm_layers += [nn.Conv2d(256, 1 * 10, kernel_size=3, padding=1)]

    loc_layers += [nn.Conv2d(256, 1 * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, 1 * num_classes, kernel_size=3, padding=1)]
    landm_layers += [nn.Conv2d(256, 1 * 10, kernel_size=3, padding=1)]
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output


    
class Slim_a5_FPN(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_a5_FPN, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 128, 2)
    self.conv10 = conv_dw(128, 128, 1)
    self.conv11 = conv_dw(128, 128, 1)

    self.conv12 = conv_dw(128, 256, 2)
    self.conv13 = conv_dw(256, 256, 1)

    self.conv14 = conv_dw(256, 256, 2)
    self.conv15 = conv_dw(256, 256, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 256, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.fpn = FPN5([64, 128, 256, 256, 256], 64)
    self.loc, self.conf, self.landm = self.multibox(self.num_classes)

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]
    
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    detections = self.fpn(detections)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output


class Slim_a5_FPN2(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_a5_FPN2, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv3 = conv_dw(16, 32, 2)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 128, 2)
    self.conv10 = conv_dw(128, 128, 1)
    self.conv11 = conv_dw(128, 128, 1)

    self.conv12 = conv_dw(128, 256, 2)
    self.conv13 = conv_dw(256, 256, 1)

    self.conv14 = conv_dw(256, 256, 2)
    self.conv15 = conv_dw(256, 256, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 256, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.fpn = FPN5([64, 128, 256, 256, 256], 64)
    self.loc, self.conf, self.landm = self.multibox(self.num_classes)

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 1 * 10, kernel=3, pad=1)]
    
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x3 = self.conv3(x1)
    x5 = self.conv5(x3)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    detections = self.fpn(detections)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

class Slim_c64_FPN(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_c64_FPN, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 64, 2)
    self.conv10 = conv_dw(64, 64, 1)
    self.conv11 = conv_dw(64, 64, 1)

    self.conv12 = conv_dw(64, 64, 2)
    self.conv13 = conv_dw(64, 64, 1)

    self.conv14 = conv_dw(64, 64, 2)
    self.conv15 = conv_dw(64, 64, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 64, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.fpn = FPN5([64, 64, 64, 64, 64], 64)
    self.loc, self.conf, self.landm = self.multibox(self.num_classes)

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 3 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 3 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 3 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]
    
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    detections = self.fpn(detections)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

    
class Slim_FPN(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_FPN, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 128, 2)
    self.conv10 = conv_dw(128, 128, 1)
    self.conv11 = conv_dw(128, 128, 1)

    self.conv12 = conv_dw(128, 256, 2)
    self.conv13 = conv_dw(256, 256, 1)

    self.conv14 = conv_dw(256, 256, 2)
    self.conv15 = conv_dw(256, 256, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 256, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.fpn = FPN5([64, 128, 256, 256, 256], 64)
    self.loc, self.conf, self.landm = self.multibox(self.num_classes)

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 3 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 3 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 3 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]
    
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    detections = self.fpn(detections)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

class Slim_FPN_dw256(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_FPN_dw256, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 128, 2)
    self.conv10 = conv_dw(128, 128, 1)
    self.conv11 = conv_dw(128, 128, 1)

    self.conv12 = conv_dw(128, 256, 2)
    self.conv13 = conv_dw(256, 256, 1)

    self.conv14 = conv_dw(256, 256, 2)
    self.conv15 = conv_dw(256, 256, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 256, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.fpn = FPN_dw([64, 128, 256, 256, 256], 256)
    self.loc, self.conf, self.landm = self.multibox(self.num_classes)

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(256, 3 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 3 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 3 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 2 * 10, kernel=3, pad=1)]
    
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    detections = self.fpn(detections)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

class Slim_a5_FPN_dw256(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_a5_FPN_dw256, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 128, 2)
    self.conv10 = conv_dw(128, 128, 1)
    self.conv11 = conv_dw(128, 128, 1)

    self.conv12 = conv_dw(128, 256, 2)
    self.conv13 = conv_dw(256, 256, 1)

    self.conv14 = conv_dw(256, 256, 2)
    self.conv15 = conv_dw(256, 256, 1)

    self.conv16 = nn.Sequential(
      nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1),
      nn.ReLU(inplace=True),
      depth_conv2d(64, 256, kernel=3, stride=2, pad=1),
      nn.ReLU(inplace=True)
    )
    self.fpn = FPN5_dw([64, 128, 256, 256, 256], 256)
    self.loc, self.conf, self.landm = self.multibox(self.num_classes)

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(256, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 1 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(256, 1 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(256, 1 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(256, 1 * 10, kernel=3, pad=1)]
    
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    detections = self.fpn(detections)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

class Slim_64_FPNx_dw64(nn.Module):
  def __init__(self, cfg = None, phase = 'train'):
    """
    :param cfg:  Network related settings.
    :param phase: train or test.
    """
    super(Slim_64_FPNx_dw64, self).__init__()
    self.phase = phase
    self.num_classes = 2

    self.conv1 = conv_bn(3, 16, 2)
    self.conv2 = conv_dw(16, 32, 1)
    self.conv3 = conv_dw(32, 32, 2)
    self.conv4 = conv_dw(32, 32, 1)
    self.conv5 = conv_dw(32, 64, 2)
    self.conv6 = conv_dw(64, 64, 1)
    self.conv7 = conv_dw(64, 64, 1)
    self.conv8 = conv_dw(64, 64, 1)

    self.conv9 = conv_dw(64, 64, 2)
    self.conv10 = conv_dw(64, 64, 1)
    self.conv11 = nn.Sequential(
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1)
    )

    self.conv12 = conv_dw(64, 64, 2)
    self.conv13 = nn.Sequential(
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1)
    )

    self.conv14 = conv_dw(64, 64, 2)
    self.conv15 = nn.Sequential(
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1)
    )

    self.conv16 = nn.Sequential(
        conv_dw(64, 64, 2),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1),
        conv_dw(64, 64, 1)
    )
    self.fpn = FPN5_dw([64, 64, 64, 64, 64], 64)
    self.loc, self.conf, self.landm = self.multibox(self.num_classes)

  def multibox(self, num_classes):
    loc_layers = []
    conf_layers = []
    landm_layers = []
    loc_layers += [depth_conv2d(64, 3 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 3 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 3 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]

    loc_layers += [depth_conv2d(64, 2 * 4, kernel=3, pad=1)]
    conf_layers += [depth_conv2d(64, 2 * num_classes, kernel=3, pad=1)]
    landm_layers += [depth_conv2d(64, 2 * 10, kernel=3, pad=1)]
    
    return nn.Sequential(*loc_layers), nn.Sequential(*conf_layers), nn.Sequential(*landm_layers)


  def forward(self,inputs):
    detections = list()
    loc = list()
    conf = list()
    landm = list()

    x1 = self.conv1(inputs)
    x2 = self.conv2(x1)
    x3 = self.conv3(x2)
    x4 = self.conv4(x3)
    x5 = self.conv5(x4)
    x6 = self.conv6(x5)
    x7 = self.conv7(x6)
    x8 = self.conv8(x7)
    detections.append(x8)

    x9 = self.conv9(x8)
    x10 = self.conv10(x9)
    x11 = self.conv11(x10)
    detections.append(x11)

    x12 = self.conv12(x11)
    x13 = self.conv13(x12)
    detections.append(x13)

    x14 = self.conv14(x13)
    x15 = self.conv15(x14)
    detections.append(x15)

    x16 = self.conv14(x15)
    detections.append(x16)

    detections = self.fpn(detections)

    for (x, l, c, lam) in zip(detections, self.loc, self.conf, self.landm):
      loc.append(l(x).permute(0, 2, 3, 1).contiguous())
      conf.append(c(x).permute(0, 2, 3, 1).contiguous())
      landm.append(lam(x).permute(0, 2, 3, 1).contiguous())

    bbox_regressions = torch.cat([o.view(o.size(0), -1, 4) for o in loc], 1)
    classifications = torch.cat([o.view(o.size(0), -1, 2) for o in conf], 1)
    ldm_regressions = torch.cat([o.view(o.size(0), -1, 10) for o in landm], 1)

    if self.phase == 'train':
      output = (bbox_regressions, classifications, ldm_regressions)
    else:
      output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
    return output

### B

In [0]:
import torch
import torch.nn as nn
import torchvision.models.detection.backbone_utils as backbone_utils
import torchvision.models._utils as _utils
import torch.nn.functional as F
from collections import OrderedDict

from models.net import MobileNetV1 as MobileNetV1
from models.net import FPN as FPN
from models.net import SSH as SSH



class ClassHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(ClassHead,self).__init__()
        self.num_anchors = num_anchors
        self.conv1x1 = nn.Conv2d(inchannels,self.num_anchors*2,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()
        
        return out.view(out.shape[0], -1, 2)

class BboxHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(BboxHead,self).__init__()
        self.conv1x1 = nn.Conv2d(inchannels,num_anchors*4,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()

        return out.view(out.shape[0], -1, 4)

class LandmarkHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(LandmarkHead,self).__init__()
        self.conv1x1 = nn.Conv2d(inchannels,num_anchors*10,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()

        return out.view(out.shape[0], -1, 10)

class MnetFixedA0(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(MnetFixedA0,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']
        self.fpn = FPN(in_channels_list,out_channels)
        self.ssh1 = SSH(out_channels, out_channels)
        self.ssh2 = SSH(out_channels, out_channels)
        self.ssh3 = SSH(out_channels, out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=cfg['out_channel'])

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,1 if i == 0 else anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,1 if i == 0 else anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,1 if i == 0 else anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        # SSH
        feature1 = self.ssh1(fpn[0])
        feature2 = self.ssh2(fpn[1])
        feature3 = self.ssh3(fpn[2])
        features = [feature1, feature2, feature3]

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output

        
class MnetNoSsh(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(MnetNoSsh,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']
        self.fpn = FPN(in_channels_list,out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=cfg['out_channel'])

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        features = fpn

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output

class MnetSSDFixedA0(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(MnetSSDFixedA0,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=in_channels_list)
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=in_channels_list)
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=in_channels_list)

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels[i],1 if i == 0 else anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels[i],1 if i == 0 else anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels[i],1 if i == 0 else anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        features = list(out.values())

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output

      
class MnetSSD(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(MnetSSD,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=in_channels_list)
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=in_channels_list)
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=in_channels_list)

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels[i],anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels[i],anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels[i],anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        features = list(out.values())

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output

class MnetNorm(RetinaFace):
  def forward(self, inputs):
    normalized_inputs = inputs / 255.0
    return super().forward(normalized_inputs)



class MobileNetV1x(nn.Module):
    def __init__(self):
        super(MobileNetV1x, self).__init__()
        self.stage1 = nn.Sequential(
            conv_bn(3, 8, 2),    # 3
            conv_dw(8, 16, 1),   # 7
            conv_dw(16, 32, 2),  # 11
            conv_dw(32, 32, 1),  # 19
            conv_dw(32, 64, 2),  # 27
            conv_dw(64, 64, 1),  # 43
            conv_dw(64, 64, 1),  # 43
            conv_dw(64, 64, 1),  # 43
            conv_dw(64, 64, 1),  # 43
        )
        self.stage2 = nn.Sequential(
            conv_dw(64, 128, 2),  # 43 + 16 = 59
            conv_dw(128, 128, 1), # 59 + 32 = 91
            conv_dw(128, 128, 1), # 91 + 32 = 123
            conv_dw(128, 128, 1), # 123 + 32 = 155
            conv_dw(128, 128, 1), # 155 + 32 = 187
            conv_dw(128, 128, 1), # 187 + 32 = 219
            conv_dw(128, 128, 1), # 187 + 32 = 219
            conv_dw(128, 128, 1), # 187 + 32 = 219
        )
        self.stage3 = nn.Sequential(
            conv_dw(128, 256, 2), # 219 +3 2 = 241
            conv_dw(256, 256, 1), # 241 + 64 = 301
            conv_dw(256, 256, 1), # 241 + 64 = 301
        )
        self.avg = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256, 1000)

    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.avg(x)
        # x = self.model(x)
        x = x.view(-1, 256)
        x = self.fc(x)
        return x

class MnetX(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(MnetX,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1x()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']
        self.fpn = FPN(in_channels_list,out_channels)
        self.ssh1 = SSH(out_channels, out_channels)
        self.ssh2 = SSH(out_channels, out_channels)
        self.ssh3 = SSH(out_channels, out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=cfg['out_channel'])

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        # SSH
        feature1 = self.ssh1(fpn[0])
        feature2 = self.ssh2(fpn[1])
        feature3 = self.ssh3(fpn[2])
        features = [feature1, feature2, feature3]

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output


class MobileNetV1_128(nn.Module):
    def __init__(self):
        super(MobileNetV1_128, self).__init__()
        self.stage1 = nn.Sequential(
            conv_bn(3, 16, 2),    # 3
            conv_dw(16, 32, 1),   # 7
            conv_dw(32, 64, 2),  # 11
            conv_dw(64, 64, 1),  # 19
            conv_dw(64, 64, 2),  # 27
            conv_dw(64, 128, 1),  # 43
        )
        self.stage2 = nn.Sequential(
            conv_dw(128, 128, 2),  # 43 + 16 = 59
            conv_dw(128, 128, 1), # 59 + 32 = 91
            conv_dw(128, 128, 1), # 91 + 32 = 123
            conv_dw(128, 128, 1), # 123 + 32 = 155
            conv_dw(128, 128, 1), # 155 + 32 = 187
            conv_dw(128, 128, 1), # 187 + 32 = 219
        )
        self.stage3 = nn.Sequential(
            conv_dw(128, 256, 2), # 219 +3 2 = 241
            conv_dw(256, 256, 1), # 241 + 64 = 301
        )
        self.avg = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256, 1000)

    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.avg(x)
        # x = self.model(x)
        x = x.view(-1, 256)
        x = self.fc(x)
        return x
class Mnet_128(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(Mnet_128,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1_128()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
        ]
        out_channels = cfg['out_channel']
        self.fpn = FPN(in_channels_list,out_channels)
        self.ssh1 = SSH(out_channels, out_channels)
        self.ssh2 = SSH(out_channels, out_channels)
        self.ssh3 = SSH(out_channels, out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=cfg['out_channel'])

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        # SSH
        feature1 = self.ssh1(fpn[0])
        feature2 = self.ssh2(fpn[1])
        feature3 = self.ssh3(fpn[2])
        features = [feature1, feature2, feature3]

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output


class MobileNetV1_s1L(nn.Module):
    def __init__(self):
        super(MobileNetV1_s1L, self).__init__()
        self.stage1 = nn.Sequential(
            conv_bn(3, 16, 2),    # 3
            conv_dw(16, 32, 1),   # 7
            conv_dw(32, 64, 2),  # 11
            conv_dw(64, 64, 1),  # 19
            conv_dw(64, 64, 2),  # 27
            conv_dw(64, 64, 1),  # 43
        )
        self.stage2 = nn.Sequential(
            conv_dw(64, 128, 2),  # 43 + 16 = 59
            conv_dw(128, 128, 1), # 59 + 32 = 91
            conv_dw(128, 128, 1), # 91 + 32 = 123
            conv_dw(128, 128, 1), # 123 + 32 = 155
            conv_dw(128, 128, 1), # 155 + 32 = 187
            conv_dw(128, 128, 1), # 187 + 32 = 219
        )
        self.stage3 = nn.Sequential(
            conv_dw(128, 256, 2), # 219 +3 2 = 241
            conv_dw(256, 256, 1), # 241 + 64 = 301
        )
        self.avg = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256, 1000)

    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.avg(x)
        # x = self.model(x)
        x = x.view(-1, 256)
        x = self.fc(x)
        return x
        
class Mnet_s1L(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(Mnet_s1L,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1_s1L()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']
        self.fpn = FPN(in_channels_list,out_channels)
        self.ssh1 = SSH(out_channels, out_channels)
        self.ssh2 = SSH(out_channels, out_channels)
        self.ssh3 = SSH(out_channels, out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=cfg['out_channel'])

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        # SSH
        feature1 = self.ssh1(fpn[0])
        feature2 = self.ssh2(fpn[1])
        feature3 = self.ssh3(fpn[2])
        features = [feature1, feature2, feature3]

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output

class MobileNetV1_s0_160(nn.Module):
    def __init__(self):
        super(MobileNetV1_s0_160, self).__init__()
        self.stage1 = nn.Sequential(
            conv_bn(3, 8, 2),    # 3
            conv_dw(8, 16, 1),   # 7
            conv_dw(16, 32, 2),  # 11
            conv_dw(32, 32, 1),  # 19
            conv_dw(32, 64, 1),  # 27
            conv_dw(64, 64, 1),  # 43
        )
        self.stage2 = nn.Sequential(
            conv_dw(64, 128, 2),  # 43 + 16 = 59
            conv_dw(128, 128, 1), # 59 + 32 = 91
            conv_dw(128, 128, 1), # 91 + 32 = 123
            conv_dw(128, 128, 1), # 123 + 32 = 155
            conv_dw(128, 128, 1), # 155 + 32 = 187
            conv_dw(128, 128, 1), # 187 + 32 = 219
        )
        self.stage3 = nn.Sequential(
            conv_dw(128, 256, 2), # 219 +3 2 = 241
            conv_dw(256, 256, 1), # 241 + 64 = 301
        )
        self.avg = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256, 1000)

    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.avg(x)
        # x = self.model(x)
        x = x.view(-1, 256)
        x = self.fc(x)
        return x
class Mnet_s0_160(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(Mnet_s0_160,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1_s0_160()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']
        self.fpn = FPN(in_channels_list,out_channels)
        self.ssh1 = SSH(out_channels, out_channels)
        self.ssh2 = SSH(out_channels, out_channels)
        self.ssh3 = SSH(out_channels, out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=cfg['out_channel'])

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        # SSH
        feature1 = self.ssh1(fpn[0])
        feature2 = self.ssh2(fpn[1])
        feature3 = self.ssh3(fpn[2])
        features = [feature1, feature2, feature3]

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output

## create_ULFGFD_plus

In [0]:

torch.set_default_tensor_type('torch.FloatTensor')
def create_ULFGFD_plus(cpu = False, nms_threshold =0.3, target_size = 640, model='mnet', weights_path=None):
  if model == "mnet":
    weights_file = 'mobilenet0.25_Final.pth'
    cfg = cfg_mnet
    net = RetinaFace(cfg = cfg, phase = 'test')
  elif model == "slim":
    weights_file = 'slim_Final.pth'
    cfg = cfg_slim
    net = Slim(cfg = cfg, phase = 'test')
  elif model == "rfb":
    weights_file = 'RBF_Final.pth'
    cfg = cfg_rfb
    net = RFB(cfg = cfg, phase = 'test')
  elif model == "slim_c64":
    cfg = cfg_slim
    net = Slim_c64(cfg = cfg, phase = 'test')
  elif model == "slim_c64_a5":
    cfg = {
      'min_sizes': [[16], [32], [64], [128], [256]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0
    }
    net = Slim_c64_a5(cfg = cfg, phase = 'test')
  elif model == "slim_a5":
    cfg = {
      'min_sizes': [[16], [32], [64], [128], [256]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0
    }
    net = Slim_a5(cfg = cfg, phase = 'test')
  elif model == "slim_a5_fpn":
    cfg = {
      'min_sizes': [[16], [32], [64], [128], [256]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0
    }
    net = Slim_a5_FPN(cfg = cfg, phase = 'test')
  elif model == "slim_a5_fpn2":
    cfg = {
      'min_sizes': [[16], [32], [64], [128], [256]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0
    }
    net = Slim_a5_FPN2(cfg = cfg, phase = 'test')
  elif model == "slim_c64_fpn":
    cfg = {
      'min_sizes': [[10, 16, 24], [32, 48], [64, 96], [128, 192], [256, 384]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0
    }
    net = Slim_c64_FPN(cfg = cfg, phase = 'test')
  elif model == "slim_fpn":
    cfg = {
      'min_sizes': [[10, 16, 24], [32, 48], [64, 96], [128, 192], [256, 384]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0
    }
    net = Slim_FPN(cfg = cfg, phase = 'test')
  elif model == "slim_fpn_dw256":
    cfg = {
      'min_sizes': [[10, 16, 24], [32, 48], [64, 96], [128, 192], [256, 384]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0,
      'gpu_train': True,
      'batch_size': 32,
      'ngpu': 1,
      'epoch': 250,
      'decay1': 190,
      'decay2': 220
    }
    net = Slim_FPN_dw256()
  elif model == "slim_a5_fpn_dw256":
    cfg = {
      'min_sizes': [[16], [32], [64], [128], [256]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0,
      'gpu_train': True,
      'batch_size': 32,
      'ngpu': 1,
      'epoch': 250,
      'decay1': 190,
      'decay2': 220
    }
    net = Slim_a5_FPN_dw256()
  elif model == "slim_64_fpnx_dw64":
    cfg = {
      'min_sizes': [[10, 16, 24], [32, 48], [64, 96], [128, 192], [256, 384]],
      'steps': [8, 16, 32, 64, 128],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0,
      'gpu_train': True,
      'batch_size': 32,
      'ngpu': 1,
      'epoch': 250,
      'decay1': 190,
      'decay2': 220
    }
    net = Slim_64_FPNx_dw64()
  elif model == "mnet_fixed_a0":
    cfg = {
      'name': 'mobilenet0.25',
      'min_sizes': [[16], [32, 64], [128, 256]],
      'steps': [8, 16, 32],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0,
      'gpu_train': True,
      'batch_size': 32,
      'ngpu': 1,
      'epoch': 250,
      'decay1': 190,
      'decay2': 220,
      'pretrain': False,
      'return_layers': {'stage1': 1, 'stage2': 2, 'stage3': 3},
      'in_channel': 32,
      'out_channel': 64
    }
    net = MnetFixedA0(cfg=cfg)
  elif model == "mnet_no_ssh":
    cfg = cfg_mnet
    net = MnetNoSsh(cfg=cfg_mnet)
  elif model == "mnet_ssd_fixed_a0":
    cfg = {
      'name': 'mobilenet0.25',
      'min_sizes': [[16], [32, 64], [128, 256]],
      'steps': [8, 16, 32],
      'variance': [0.1, 0.2],
      'clip': False,
      'loc_weight': 2.0,
      'gpu_train': True,
      'batch_size': 32,
      'ngpu': 1,
      'epoch': 250,
      'decay1': 190,
      'decay2': 220,
      'pretrain': False,
      'return_layers': {'stage1': 1, 'stage2': 2, 'stage3': 3},
      'in_channel': 32,
      'out_channel': 64
    }
    net = MnetSSDFixedA0(cfg=cfg)
  elif model == "mnet_ssd":
    cfg = cfg_mnet
    net = MnetSSD(cfg=cfg)
  elif model == "mnet_norm":
    cfg = cfg_mnet
    net = MnetNorm(cfg=cfg)
  elif model == "mnet_x":
    cfg = cfg_mnet
    net = MnetX(cfg=cfg)
  elif model == "mnet_128":
    cfg = cfg_mnet.copy()
    cfg['in_channel'] = 64
    cfg['out_channel'] = 128
    net = Mnet_128(cfg=cfg)
  elif model == "mnet_s1l":
    cfg = cfg_mnet
    net = Mnet_s1L(cfg=cfg)
  elif model == "mnet_s0_160":
    cfg = cfg_mnet.copy()
    cfg['steps'] = [4, 8, 16]
    net = Mnet_s0_160(cfg=cfg)

  #device = torch.device("cpu" if cpu else "cuda")
  if weights_path is None:
    weights_path = './Face-Detector-1MB-with-landmark/weights/' + weights_file 
  net = load_model(net, weights_path, True)
  net.eval()
  def detect(img, min_score):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    max_size = target_size
    im_shape = img.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])
    resize = float(target_size) / float(im_size_min)
    # prevent bigger axis from being more than max_size:
    if np.round(resize * im_size_max) > max_size:
        resize = float(max_size) / float(im_size_max)
    if resize != 1:
        img = cv2.resize(img, None, None, fx=resize, fy=resize, interpolation=cv2.INTER_LINEAR)
    im_height, im_width, _ = img.shape
    scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
    #img -= np.array([104, 117, 123]).astype('uint8')
    #img = img.transpose(2, 0, 1).astype('float32')
    img = np.moveaxis(img, 2, 0).astype('float32')
    img = torch.from_numpy(img).unsqueeze(0)
    loc, conf, landms = net(img)

    #print(im_height, im_width, img.shape, resize)

    priorbox = PriorBox(cfg, image_size=(im_height, im_width))
    priors = priorbox.forward()
    prior_data = priors.data
    boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
    boxes = boxes * scale
    boxes = boxes.cpu().numpy()
    scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
    landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance'])
    scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                            img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                            img.shape[3], img.shape[2]])
    #scale1 = scale1.to(device)
    landms = landms * scale1
    landms = landms.cpu().numpy()

    # ignore low scores
    inds = np.where(scores > min_score)[0]
    boxes = boxes[inds]
    landms = landms[inds]
    scores = scores[inds]

    # keep top-K before NMS
    order = scores.argsort()[::-1][:100]
    boxes = boxes[order]
    landms = landms[order]
    scores = scores[order]

    # do NMS
    dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
    keep = py_cpu_nms(dets, nms_threshold)
    # keep = nms(dets, args.nms_threshold,force_cpu=args.cpu)
    dets = dets[keep, :]
    landms = landms[keep]


    out_faces = []
    for face in dets:
      x0, y0, x1, y1, s = face[0:5]
      x0, x1 = np.array([x0, x1]) / im_width
      y0, y1 = np.array([y0, y1]) / im_height
      out_faces.append([x0, y0, x1-x0, y1-y0, s])
    return out_faces
  return detect

# Plot

In [0]:
from graphviz import Digraph
import re
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Variable
import torchvision.models as models
from torchviz import make_dot
torch.set_default_tensor_type('torch.FloatTensor')

weights_file = 'mnet_260_640_80_epoch_249.pth'
cfg = cfg_mnet
net = RetinaFace(cfg = cfg, phase = 'test')
net = load_model(net, weights_file, True)
#net.load_state_dict(torch.load('./mobilenet0.25_Final.pth'))
y = net(Variable(torch.from_numpy(np.zeros([3, 640, 640]).astype('float32')).unsqueeze(0)))

g = make_dot(y)
g.view()

# Extract Weights Mnet

In [0]:
import json
weights_file = 'mnet_260_640_80_epoch_249.pth'
cfg = cfg_mnet
net = RetinaFace(cfg = cfg, phase = 'test')
net = load_model(net, weights_file, True)


layers = []
layer_names = []

# BODY
for stage, module in net.body.items():
  for idx, seq in enumerate(module):
    module_prefix = "mnet/{}/conv_{}".format(stage.replace('stage', 'stage_'), idx)
    if stage == 'stage1' and idx == 0:
      layer_names.append("{}/filters".format(module_prefix))
      layer_names.append("{}/batch_norm".format(module_prefix))
    else:
      layer_names.append("{}/depthwise_filter".format(module_prefix))
      layer_names.append("{}/batch_norm_depthwise_conv".format(module_prefix))
      layer_names.append("{}/pointwise_filter".format(module_prefix))
      layer_names.append("{}/batch_norm_pointwise_conv".format(module_prefix))
    for l in seq:
      if hasattr(l, 'weight'):
        layers.append(l)

# FPN
for ls in [net.fpn.output3, net.fpn.output2, net.fpn.output1, net.fpn.merge2, net.fpn.merge1]:
  for l in ls:
    if hasattr(l, 'weight'):
      layers.append(l)
for name in ['conv_shrink_stage_3', 'conv_shrink_stage_2', 'conv_shrink_stage_1', 'conv_anti_aliasing_stage_2', 'conv_anti_aliasing_stage_1']:
  layer_names.append("fpn/{}/filters".format(name))
  layer_names.append("fpn/{}/batch_norm".format(name))

# SSH
for idx, ssh in enumerate([net.ssh1, net.ssh2, net.ssh3]):
  for module in [ssh.conv3X3, ssh.conv5X5_1, ssh.conv5X5_2, ssh.conv7X7_2, ssh.conv7x7_3]:
    for l in module:
      if hasattr(l, 'weight'):
        layers.append(l)
  for name in ['conv_3x3', 'conv_5x5_1', 'conv_5x5_2', 'conv_7x7_1', 'conv_7x7_2']:
    layer_names.append("ssh_{}/{}/filters".format(idx + 1, name))
    layer_names.append("ssh_{}/{}/batch_norm".format(idx + 1, name))

# HEADS
for idx, l in enumerate(net.ClassHead):
  layers.append(l.conv1x1)
  layer_names.append("conv_score_stage_{}/filters".format(idx + 1))
for idx, l in enumerate(net.BboxHead):
  layers.append(l.conv1x1)
  layer_names.append("conv_box_stage_{}/filters".format(idx + 1))
for idx, l in enumerate(net.LandmarkHead):
  layers.append(l.conv1x1)
  layer_names.append("conv_landmarks_stage_{}/filters".format(idx + 1))

print(net.ClassHead)
print(net.BboxHead)
print(net.LandmarkHead)

weight_names = []
weights_array = []
for layer, name in zip(layers, layer_names):
  if hasattr(layer, 'running_mean'):
    weights_array.append(layer.running_mean.detach().numpy())
    weights_array.append(layer.running_var.detach().numpy())
    weights_array.append(layer.weight.detach().numpy())
    weights_array.append(layer.bias.detach().numpy())
    weight_names.append(name + '/mean')
    weight_names.append(name + '/variance')
    weight_names.append(name + '/scale')
    weight_names.append(name + '/offset')
  else:
    weights = layer.weight.detach().numpy()
    if "depthwise_filter" in name:
      weights = np.moveaxis(np.moveaxis(weights, 0, -1), 0, -1)
    else:
      weights = np.moveaxis(np.moveaxis(weights, 1, -1), 0, -1)
    weights_array.append(weights)
    weight_names.append(name)

manifest_weights = []
for weights, name in zip(weights_array, weight_names):
  print(name, weights.shape, np.sum(np.abs(weights)))
  manifest_weights.append({
    "name": name, "shape": weights.shape, "dtype":"float32"
  })

concated_weights = np.array([])
for weights in weights_array:
  concated_weights = np.append(concated_weights, weights.flatten())
  print(concated_weights.shape, np.sum(np.abs(concated_weights)))

#print(concated_weights.shape)
shard = 'mnet_260_640_80_epoch_249-shard1.bin'
concated_weights = concated_weights.astype('float32')
concated_weights.tofile(shard)
print(concated_weights[0:216])
print(concated_weights.shape)
manifest = [{
  "weights": manifest_weights,
  "paths": [shard]
}]

with open('mnet_260_640_80_epoch_249.json', 'w') as file:
  file.write(json.dumps(manifest))



In [0]:
[{"weights":
  [{"name":"entry_flow/conv_in/filters","shape":[3,3,3,32],"dtype":"float32",
    "quantization":{"dtype":"uint8","scale":0.005431825039433498,"min":-0.7441600304023892}},
   {"name":"entry_flow/conv_in/bias","shape":[32],"dtype":"float32"},
   {"name":"entry_flow/reduction_block_0/separable_conv0/depthwise_filter","shape":[3,3,32,1],"dtype":"float32",
    "quantization":{"dtype":"uint8","scale":0.005691980614381678,"min":-0.6090419257388395}},
   {"name":"entry_flow/reduction_block_0/separable_conv0/pointwise_filter","shape":[1,1,32,64],"dtype":"float32",
    "quantization":{"dtype":"uint8","scale":0.009089225881239947,"min":-1.1179747833925135}},
   {"name":"entry_flow/reduction_block_0/separable_conv0/bias","shape":[64],"dtype":"float32"},
   {"name":"entry_flow/reduction_block_0/separable_conv1/depthwise_filter","shape":[3,3,64,1],"dtype":"float32",
    "quantization":{"dtype":"uint8","scale":0.00683894624897078,"min":-0.8138346036275228}},
   {"name":"entry_flow/reduction_block_0/separable_conv1/pointwise_filter","shape":[1,1,64,64],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.011632566358528886,"min":-1.3028474321552352}},{"name":"entry_flow/reduction_block_0/separable_conv1/bias","shape":[64],"dtype":"float32"},{"name":"entry_flow/reduction_block_0/expansion_conv/filters","shape":[1,1,32,64],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.010254812240600587,"min":-0.9229331016540528}},{"name":"entry_flow/reduction_block_0/expansion_conv/bias","shape":[64],"dtype":"float32"},{"name":"entry_flow/reduction_block_1/separable_conv0/depthwise_filter","shape":[3,3,64,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.0052509616403018725,"min":-0.6406173201168285}},{"name":"entry_flow/reduction_block_1/separable_conv0/pointwise_filter","shape":[1,1,64,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.010788509424994973,"min":-1.4564487723743214}},{"name":"entry_flow/reduction_block_1/separable_conv0/bias","shape":[128],"dtype":"float32"},{"name":"entry_flow/reduction_block_1/separable_conv1/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.00553213918910307,"min":-0.7025816770160899}},{"name":"entry_flow/reduction_block_1/separable_conv1/pointwise_filter","shape":[1,1,128,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.013602388606351965,"min":-1.6186842441558837}},{"name":"entry_flow/reduction_block_1/separable_conv1/bias","shape":[128],"dtype":"float32"},{"name":"entry_flow/reduction_block_1/expansion_conv/filters","shape":[1,1,64,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.007571851038465313,"min":-1.158493208885193}},{"name":"entry_flow/reduction_block_1/expansion_conv/bias","shape":[128],"dtype":"float32"},{"name":"middle_flow/main_block_0/separable_conv0/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.005766328409606335,"min":-0.6688940955143349}},{"name":"middle_flow/main_block_0/separable_conv0/pointwise_filter","shape":[1,1,128,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.012136116214826995,"min":-1.5776951079275094}},{"name":"middle_flow/main_block_0/separable_conv0/bias","shape":[128],"dtype":"float32"},{"name":"middle_flow/main_block_0/separable_conv1/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.004314773222979377,"min":-0.5652352922102984}},{"name":"middle_flow/main_block_0/separable_conv1/pointwise_filter","shape":[1,1,128,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.01107162026798024,"min":-1.2400214700137868}},{"name":"middle_flow/main_block_0/separable_conv1/bias","shape":[128],"dtype":"float32"},{"name":"middle_flow/main_block_0/separable_conv2/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.0036451735917259667,"min":-0.4848080876995536}},{"name":"middle_flow/main_block_0/separable_conv2/pointwise_filter","shape":[1,1,128,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.008791744942758598,"min":-1.134135097615859}},{"name":"middle_flow/main_block_0/separable_conv2/bias","shape":[128],"dtype":"float32"},{"name":"middle_flow/main_block_1/separable_conv0/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.004915751896652521,"min":-0.6095532351849126}},{"name":"middle_flow/main_block_1/separable_conv0/pointwise_filter","shape":[1,1,128,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.010868691463096469,"min":-1.3368490499608656}},{"name":"middle_flow/main_block_1/separable_conv0/bias","shape":[128],"dtype":"float32"},{"name":"middle_flow/main_block_1/separable_conv1/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.005010117269029804,"min":-0.6012140722835765}},{"name":"middle_flow/main_block_1/separable_conv1/pointwise_filter","shape":[1,1,128,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.010311148213405235,"min":-1.3816938605963016}},{"name":"middle_flow/main_block_1/separable_conv1/bias","shape":[128],"dtype":"float32"},{"name":"middle_flow/main_block_1/separable_conv2/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.004911523706772748,"min":-0.7367285560159123}},{"name":"middle_flow/main_block_1/separable_conv2/pointwise_filter","shape":[1,1,128,128],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.008976466047997568,"min":-1.2207993825276693}},{"name":"middle_flow/main_block_1/separable_conv2/bias","shape":[128],"dtype":"float32"},{"name":"exit_flow/reduction_block/separable_conv0/depthwise_filter","shape":[3,3,128,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.005074804436926748,"min":-0.7104726211697447}},{"name":"exit_flow/reduction_block/separable_conv0/pointwise_filter","shape":[1,1,128,256],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.011453078307357489,"min":-1.4545409450344011}},{"name":"exit_flow/reduction_block/separable_conv0/bias","shape":[256],"dtype":"float32"},{"name":"exit_flow/reduction_block/separable_conv1/depthwise_filter","shape":[3,3,256,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.007741751390344957,"min":-1.1380374543807086}},{"name":"exit_flow/reduction_block/separable_conv1/pointwise_filter","shape":[1,1,256,256],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.011347713189966538,"min":-1.497898141075583}},{"name":"exit_flow/reduction_block/separable_conv1/bias","shape":[256],"dtype":"float32"},{"name":"exit_flow/reduction_block/expansion_conv/filters","shape":[1,1,128,256],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.006717281014311547,"min":-0.8329428457746318}},{"name":"exit_flow/reduction_block/expansion_conv/bias","shape":[256],"dtype":"float32"},{"name":"exit_flow/separable_conv/depthwise_filter","shape":[3,3,256,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.0027201742518181892,"min":-0.3237007359663645}},{"name":"exit_flow/separable_conv/pointwise_filter","shape":[1,1,256,512],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.010076364348916447,"min":-1.330080094056971}},{"name":"exit_flow/separable_conv/bias","shape":[512],"dtype":"float32"},{"name":"fc/age/weights","shape":[512,1],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.008674054987290326,"min":-1.2664120281443876}},{"name":"fc/age/bias","shape":[1],"dtype":"float32"},{"name":"fc/gender/weights","shape":[512,2],"dtype":"float32","quantization":{"dtype":"uint8","scale":0.0029948226377075793,"min":-0.34140978069866407}},{"name":"fc/gender/bias","shape":[2],"dtype":"float32"}
   ],"paths":["age_gender_model-shard1"]}]

# Debug Forward

In [0]:
import torch
import torch.nn as nn
import torchvision.models.detection.backbone_utils as backbone_utils
import torchvision.models._utils as _utils
import torch.nn.functional as F
from collections import OrderedDict

from models.net import MobileNetV1 as MobileNetV1
from models.net import FPN as FPN
from models.net import SSH as SSH



class ClassHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(ClassHead,self).__init__()
        self.num_anchors = num_anchors
        self.conv1x1 = nn.Conv2d(inchannels,self.num_anchors*2,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()
        
        return out.view(out.shape[0], -1, 2)

class BboxHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(BboxHead,self).__init__()
        self.conv1x1 = nn.Conv2d(inchannels,num_anchors*4,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()

        return out.view(out.shape[0], -1, 4)

class LandmarkHead(nn.Module):
    def __init__(self,inchannels=512,num_anchors=3):
        super(LandmarkHead,self).__init__()
        self.conv1x1 = nn.Conv2d(inchannels,num_anchors*10,kernel_size=(1,1),stride=1,padding=0)

    def forward(self,x):
        out = self.conv1x1(x)
        out = out.permute(0,2,3,1).contiguous()

        return out.view(out.shape[0], -1, 10)

class RetinaFace(nn.Module):
    def __init__(self, cfg = None, phase = 'train'):
        """
        :param cfg:  Network related settings.
        :param phase: train or test.
        """
        super(RetinaFace,self).__init__()
        self.phase = phase
        backbone = None
        if cfg['name'] == 'mobilenet0.25':
            backbone = MobileNetV1()
            if cfg['pretrain']:
                checkpoint = torch.load("./weights/mobilenetV1X0.25_pretrain.tar", map_location=torch.device('cpu'))
                from collections import OrderedDict
                new_state_dict = OrderedDict()
                for k, v in checkpoint['state_dict'].items():
                    name = k[7:]  # remove module.
                    new_state_dict[name] = v
                # load params
                backbone.load_state_dict(new_state_dict)
        elif cfg['name'] == 'Resnet50':
            import torchvision.models as models
            backbone = models.resnet50(pretrained=cfg['pretrain'])

        self.body = _utils.IntermediateLayerGetter(backbone, cfg['return_layers'])
        in_channels_stage2 = cfg['in_channel']
        in_channels_list = [
            in_channels_stage2 * 2,
            in_channels_stage2 * 4,
            in_channels_stage2 * 8,
        ]
        out_channels = cfg['out_channel']
        self.fpn = FPN(in_channels_list,out_channels)
        self.ssh1 = SSH(out_channels, out_channels)
        self.ssh2 = SSH(out_channels, out_channels)
        self.ssh3 = SSH(out_channels, out_channels)

        self.ClassHead = self._make_class_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.BboxHead = self._make_bbox_head(fpn_num=3, inchannels=cfg['out_channel'])
        self.LandmarkHead = self._make_landmark_head(fpn_num=3, inchannels=cfg['out_channel'])

    def _make_class_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        classhead = nn.ModuleList()
        for i in range(fpn_num):
            classhead.append(ClassHead(inchannels,anchor_num))
        return classhead
    
    def _make_bbox_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        bboxhead = nn.ModuleList()
        for i in range(fpn_num):
            bboxhead.append(BboxHead(inchannels,anchor_num))
        return bboxhead

    def _make_landmark_head(self,fpn_num=3,inchannels=64,anchor_num=2):
        landmarkhead = nn.ModuleList()
        for i in range(fpn_num):
            landmarkhead.append(LandmarkHead(inchannels,anchor_num))
        return landmarkhead

    def forward(self,inputs):
        out = self.body(inputs)

        # FPN
        fpn = self.fpn(out)

        # SSH
        feature1 = self.ssh1(fpn[0])
        feature2 = self.ssh2(fpn[1])
        feature3 = self.ssh3(fpn[2])
        features = [feature1, feature2, feature3]

        bbox_regressions = torch.cat([self.BboxHead[i](feature) for i, feature in enumerate(features)], dim=1)
        classifications = torch.cat([self.ClassHead[i](feature) for i, feature in enumerate(features)],dim=1)
        ldm_regressions = torch.cat([self.LandmarkHead[i](feature) for i, feature in enumerate(features)], dim=1)

        if self.phase == 'train':
            output = (bbox_regressions, classifications, ldm_regressions)
        else:
            output = (bbox_regressions, F.softmax(classifications, dim=-1), ldm_regressions)
        return output
   
from augment.augment import abs_coords

def draw_box(img, box, color = (255, 0, 0)):
  x, y, w, h = abs_coords(box, img)

  cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
  cv2.circle(img, (x, y), 2, (0, 0, 255), -1)
  cv2.circle(img, (x, y + h), 2, (0, 0, 255), -1)
  cv2.circle(img, (x + w, y), 2, (0, 0, 255), -1)
  cv2.circle(img, (x + w, y + h), 2, (0, 0, 255), -1)
  

#a

In [54]:
weights_file = 'mnet_260_640_80_epoch_249.pth'
cfg = cfg_mnet
net = RetinaFace(cfg = cfg, phase = 'test')
net = load_model(net, weights_file, True)
net.eval()
target_size = 640
min_score = 0.5

img = cv2.imread('./got.jpg')
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
max_size = target_size
im_shape = img.shape
im_size_min = np.min(im_shape[0:2])
im_size_max = np.max(im_shape[0:2])
resize = float(target_size) / float(im_size_min)
# prevent bigger axis from being more than max_size:
if np.round(resize * im_size_max) > max_size:
    resize = float(max_size) / float(im_size_max)
if resize != 1:
    img = cv2.resize(img, None, None, fx=resize, fy=resize, interpolation=cv2.INTER_LINEAR)
im_height, im_width, _ = img.shape
scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
#img -= np.array([104, 117, 123]).astype('uint8')
#img = img.transpose(2, 0, 1).astype('float32')
img = np.moveaxis(img, 2, 0).astype('float32')
img = torch.from_numpy(img).unsqueeze(0)
loc, conf, landms = net(img)

print(conf.shape)
conf = np.moveaxis(conf.detach().numpy(), 2, 0)
print(conf.shape)
*score, = conf
score = score[1]
print(score.shape)
loc = loc.detach().numpy()
scores = score[score > min_score]
boxes = loc[np.where(score > min_score)]
print(score.shape)
print(scores.shape)
print(boxes.shape)

for box, s in zip(boxes, scores):
  print(s, box)


!rm -rf ./check_inputs && mkdir ./check_inputs

for box in boxes:
  print(box)
  draw_box(img, box, color = (255, 0, 0))
file = './check_inputs/' + str(idx) + '.jpg'
cv2.imwrite(file, img)
display(Image(file))
    
!rm -rf ./check_inputs

Loading pretrained model from mnet_260_640_80_epoch_249.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
torch.Size([1, 16800, 2])
(2, 1, 16800)
(1, 16800)
(1, 16800)
(17,)
(17, 4)
0.9223244 [ 0.01361381  2.6572025  -1.1807798   0.20542033]
0.9625837 [ 2.3740065   0.16957198 -1.311083    0.14386286]
0.9858867 [-0.00292785  0.20133093 -1.3019291   0.15209888]
0.95992523 [-2.5032153   0.2269468  -1.2399713   0.09710193]
0.83198255 [ 2.3168056  -2.1022317  -1.0914615  -0.03128243]
0.9519877 [ 0.06264611 -2.453569   -1.2198281   0.14741793]
0.947363 [ 0.32183662  2.6478448  -1.246248    0.0826387 ]
0.82353264 [ 2.381438   -0.5199982  -2.019539   -0.73138475]
0.9790206 [ 2.4727626   0.18646555 -1.2337285   0.0735984 ]
0.99245 [ 0.21699834  0.2627897  -1.3449548  -0.04531649]
0.9835163 [-2.327283    0.23459072 -1.3965985  -0.01298298]
0.8809256 [ 0.50878936  0.22909369 -1.59382    -0.2055574 ]
0.9021648 [-2.0384395   0.21936119 -1.6624112  -0.31590834]
0.8103

TypeError: ignored